In [28]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tqdm import tqdm

import pennylane as qml
import pennylane.numpy as pnp  # PennyLane 전용 numpy

In [ ]:
df = pd.read_csv("creditcard.csv")
X = df.drop("Class", axis=1).values
y = df["Class"].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_res, y_res = rus.fit_resample(X_scaled, y)

X_train, X_test, y_train, y_test = train_test_split(
    X_res, y_res, test_size=0.3, random_state=42
)

In [ ]:
n_qubits = 4
L = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface="autograd")
def circuit(weights, features):
    # 앵글 인코딩
    for i, x in enumerate(features[:n_qubits]):
        qml.RX(x, wires=i)

    # Variational Ansatz (4층)
    param_idx = 0
    for _ in range(L):
        for w in range(n_qubits):
            qml.RX(weights[param_idx + 0], wires=w)
            qml.RY(weights[param_idx + 1], wires=w)
            qml.RZ(weights[param_idx + 2], wires=w)
            param_idx += 3
        for w in range(n_qubits - 1):
            qml.CNOT(wires=[w, w + 1])

    # Trash qubit(4번째) Z 기대값 반환
    return qml.expval(qml.PauliZ(n_qubits - 1))

num_params = 3 * n_qubits * L

In [ ]:
def cost_fn(weights, X_batch):
    expvals = []
    for sample in X_batch:
        # 1) pnp.array로 변환, 자동미분 그래프에 포함
        features = pnp.array(sample, requires_grad=False)
        # circuit(weights, features) → ⟨Z⟩
        expval = circuit(weights, features)
        expvals.append(expval)
     # 펜니레인에서 회로가 반환하는 것은 ⟨Z⟩. fidelity = (1 + ⟨Z⟩)/2
    fidelities = [(1 + e) / 2 for e in expvals]
    # reconstruction error(스칼라) = 배치 내 평균(1 - fidelity)
    errors = [1 - f for f in fidelities]
    return pnp.mean(pnp.stack(errors))  # 0차원 pnp.Array

# PennyLane 내장 옵티마이저
opt = qml.AdamOptimizer(stepsize=0.1)
epochs = 20
batch_size = 16

In [ ]:
# 옵티마이저, 하이퍼파라미터 설정
opt = qml.AdamOptimizer(stepsize=0.1)
epochs = 20
batch_size = 16

# 가중치 초기화 (PennyLane numpy로)
weights = pnp.random.randn(num_params, requires_grad=True)

for epoch in range(epochs):
    batch_iter = range(0, len(X_train), batch_size)
    for idx in tqdm(batch_iter, desc=f"Epoch {epoch+1}/{epochs}", ncols=80):        
        X_batch = X_train[idx : idx + batch_size]
        weights = opt.step(lambda w: cost_fn(w, X_batch), weights)
    train_loss = cost_fn(weights, X_train[:batch_size])
    print(f"Epoch {epoch+1:>2d}  loss = {train_loss.item():.4f}")

Epoch 1/20: 100%|███████████████████████████████| 43/43 [00:37<00:00,  1.16it/s]


Epoch  1  loss = 0.3164


Epoch 2/20: 100%|███████████████████████████████| 43/43 [00:36<00:00,  1.18it/s]


Epoch  2  loss = 0.3169


Epoch 3/20: 100%|███████████████████████████████| 43/43 [00:36<00:00,  1.16it/s]


Epoch  3  loss = 0.3043


Epoch 4/20: 100%|███████████████████████████████| 43/43 [00:37<00:00,  1.16it/s]


Epoch  4  loss = 0.3162


Epoch 5/20:  79%|████████████████████████▌      | 34/43 [00:29<00:07,  1.17it/s]

In [ ]:
def predict_pennylane(x, weights, threshold=0.6):
    # x: 일반 np.ndarray
    features = pnp.array(x, requires_grad=False)
    expval = circuit(weights, features)             # ⟨Z⟩
    p0 = (1 + expval) / 2                            # trash qubit이 |0>일 확률
    return 0 if p0 >= threshold else 1